In [2]:
import torch
import numpy as np
from stable_baselines3 import DDPG, SAC, TD3, DQN, PPO
from stable_baselines3.common.noise import NormalActionNoise

import os, shutil, sys
sys.path.append(os.path.abspath('./env'))
sys.path.append(os.path.abspath('./common'))

from env.env_move_sector import HumanMoveSectorAction
import common.mlflow_sb3_helper as mlf

In [3]:
mlflow_server = mlf.MLflowServerHelper("http://192.168.0.206:2670")

from pytz import timezone
from datetime import datetime

TZ = timezone('Europe/Moscow')
models = {'PPO':PPO, 'DQN':DQN, 'DDPG':DDPG, 'SAC':SAC, 'TD3':TD3}
activations = {'ReLU':torch.nn.ReLU, 'Tanh':torch.nn.Tanh}
device = "cuda" if torch.cuda.is_available() else "cpu"

mlflow_path = {}

In [3]:
env = HumanMoveSectorAction(target_point_rand=False, object_ignore=True, model_name='SAC')

env_render = HumanMoveSectorAction(target_point_rand=False, object_ignore=True, render_mode='rgb_array', model_name='SAC')

In [4]:
net_kwargs = {
'256_2': {  'activation': 'ReLU', 'pi': [256, 256], 'qf': [256, 256],},
'256_3': {  'activation': 'ReLU', 'pi': [256, 256, 256], 'qf': [256, 256, 256],},
'512_2': {  'activation': 'ReLU', 'pi': [512, 512], 'qf': [256, 256],},
'512_3': {  'activation': 'ReLU', 'pi': [512, 512, 512], 'qf': [512, 512, 512],},
}

In [5]:
exp_params = {
    'env_name': '',
    'algorithm_name': '',
    'exp_id': 0,
    'exp_name': f'exp_{datetime.now(TZ).strftime("%d%m_%H%M%S")}',
    'seed': 21,
    'net': net_kwargs['256_2'],
    'training': {
        'iteration_count': 1,
        'episode_count': 39000,
        'policy': 'MlpPolicy',
        'learning_rate': 0.003,
        'buffer_size': 1500000,
        'learning_starts': 100,
        'batch_size': 512,
        'tau': 0.005,
        'gamma': 0.99,
        'verbose': 0,
        'device': device,
    },
    'validation': {
        'validate_agent_every_n_eps': 20000,
        'log_interval': 10
    },
    'evaluation': {
        'episode_count': 1
    }
}

In [6]:
#!!!!! Новый эксперимент
exp_params['env_name'] = env.name() + 'kwargs'
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = mlflow_server.new_experiment(exp_name)
models_exclude = {'256_2':True, '256_3': False, '512_2': False, '512_3': False}

2024/09/12 14:45:51 INFO mlflow.tracking.fluent: Experiment with name 'env_MoveSector_IgnoreObstkwargs_exp_1209_144551' does not exist. Creating a new experiment.


In [ ]:
#!!!!! Дорасчет старого эксперимента
exp_params['env_name'] = env.name() + 'kwargs'
experiment_id = 171
exp_name = mlflow_server.get_experiment(experiment_id)
models_exclude = {'256_2':True, '256_3': True, '512_2': True, '512_3': False}
print(exp_name)

In [7]:
exp_params['exp_id'] = experiment_id

for name, kwargs in net_kwargs.items():

    if models_exclude != None and models_exclude[name] == True:
        continue

    exp_params['net'] = kwargs
    policy_kwargs = dict(activation_fn = activations[exp_params['net']['activation']],
                     net_arch = dict(pi=exp_params['net']['pi'], qf=exp_params['net']['qf']))


    print(name)
    exp_params['algorithm_name'] = 'SAC_' + name
    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))

    model = SAC(    exp_params['training']['policy'], 
                    env,
                    policy_kwargs=policy_kwargs,
                    learning_rate=exp_params['training']['learning_rate'],
                    buffer_size=exp_params['training']['buffer_size'],
                    learning_starts=exp_params['training']['learning_starts'],
                    batch_size=exp_params['training']['batch_size'],
                    tau=exp_params['training']['tau'],
                    gamma=exp_params['training']['gamma'],
                    verbose=exp_params['training']['verbose'],
                    device=device,
                    )

    art_loc, exp_name, run_id = mlflow_server.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=exp_params['algorithm_name'],
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'])

    mlflow_path[name] = f'{art_loc}/{run_id}/artifacts/{exp_name}/sb3/model.zip'



256_3


Output()

Moviepy - Building video env_MoveSector_IgnoreObstkwargs_exp_1209_144551//agent.mp4.
Moviepy - Writing video env_MoveSector_IgnoreObstkwargs_exp_1209_144551//agent.mp4



Moviepy - Done !
Moviepy - video ready env_MoveSector_IgnoreObstkwargs_exp_1209_144551//agent.mp4
512_2


Output()

Moviepy - Building video env_MoveSector_IgnoreObstkwargs_exp_1209_144551//agent.mp4.
Moviepy - Writing video env_MoveSector_IgnoreObstkwargs_exp_1209_144551//agent.mp4



Moviepy - Done !
Moviepy - video ready env_MoveSector_IgnoreObstkwargs_exp_1209_144551//agent.mp4
512_3


Output()

Moviepy - Building video env_MoveSector_IgnoreObstkwargs_exp_1209_144551//agent.mp4.
Moviepy - Writing video env_MoveSector_IgnoreObstkwargs_exp_1209_144551//agent.mp4



Moviepy - Done !
Moviepy - video ready env_MoveSector_IgnoreObstkwargs_exp_1209_144551//agent.mp4


In [13]:
mlflow_path

{'256_3': 'mlflow-artifacts:/175/9302819e4a5f4db49aa2bebc59f475dc/artifacts/env_MoveSector_IgnoreObstkwargs_exp_1209_144551/sb3/model.zip',
 '512_2': 'mlflow-artifacts:/175/8d290e86b1584f7a8c8544fc6c986af1/artifacts/env_MoveSector_IgnoreObstkwargs_exp_1209_144551/sb3/model.zip',
 '512_3': 'mlflow-artifacts:/175/9302819e4a5f4db49aa2bebc59f475dc/artifacts/env_MoveSector_IgnoreObstkwargs_exp_1209_144551/sb3/model.zip'}

In [6]:
experiment_id = 175

In [7]:
mlflow_path = { 
    #'256_2':'mlflow-artifacts:/171/4c0b7665e0244326852ad4adc895dad4/artifacts/env_MoveSector_IgnoreObstkwargs_exp_1109_183334/sb3/model.zip',
    '256_3':'mlflow-artifacts:/175/9302819e4a5f4db49aa2bebc59f475dc/artifacts/env_MoveSector_IgnoreObstkwargs_exp_1209_144551/sb3/model.zip',
    '512_2':'mlflow-artifacts:/175/8d290e86b1584f7a8c8544fc6c986af1/artifacts/env_MoveSector_IgnoreObstkwargs_exp_1209_144551/sb3/model.zip',
    '512_3':'mlflow-artifacts:/175/9302819e4a5f4db49aa2bebc59f475dc/artifacts/env_MoveSector_IgnoreObstkwargs_exp_1209_144551/sb3/model.zip',
 }

In [8]:

load_path = os.path.join(str(experiment_id))
os.mkdir(load_path)
load_path

'175'

In [9]:

loadel_models = {}

for model_name, path in mlflow_path.items():
    loadel_models[model_name] =  f'{load_path}/{model_name}/'
    mlflow_server.load_artifact(path,loadel_models[model_name])

In [10]:
env = HumanMoveSectorAction(target_point_rand=False, object_ignore=True, model_name='SAC')
env_render = HumanMoveSectorAction(target_point_rand=False, object_ignore=True, render_mode='rgb_array', model_name='SAC')
 
env.name()

'MoveSector_IgnoreObst'

In [11]:

exp_params['env_name'] = env.name()
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = mlflow_server.new_experiment(exp_name)
exp_params['exp_id'] = experiment_id
print(exp_name)
models_exclude = {'256_2':True, '256_3': False, '512_2': False, '512_3': False}

2024/09/12 16:16:02 INFO mlflow.tracking.fluent: Experiment with name 'env_MoveSector_IgnoreObst_exp_1209_161533' does not exist. Creating a new experiment.


env_MoveSector_IgnoreObst_exp_1209_161533


In [12]:
for name, model_path in loadel_models.items():
    
    if models_exclude != None and models_exclude[name] == True:
        continue

    print(name)

    exp_params['algorithm_name'] = 'SAC_' + name
    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))

    model = SAC.load(model_path + 'model.zip', env=env, device=device)
    art_loc, exp_name, run_id = mlflow_server.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=name,
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'])
    



256_3


Output()

Moviepy - Building video env_MoveSector_IgnoreObst_exp_1209_161533//agent.mp4.
Moviepy - Writing video env_MoveSector_IgnoreObst_exp_1209_161533//agent.mp4



Moviepy - Done !
Moviepy - video ready env_MoveSector_IgnoreObst_exp_1209_161533//agent.mp4
512_2


Output()

Moviepy - Building video env_MoveSector_IgnoreObst_exp_1209_161533//agent.mp4.
Moviepy - Writing video env_MoveSector_IgnoreObst_exp_1209_161533//agent.mp4



Moviepy - Done !
Moviepy - video ready env_MoveSector_IgnoreObst_exp_1209_161533//agent.mp4
512_3


Output()

Moviepy - Building video env_MoveSector_IgnoreObst_exp_1209_161533//agent.mp4.
Moviepy - Writing video env_MoveSector_IgnoreObst_exp_1209_161533//agent.mp4



Moviepy - Done !
Moviepy - video ready env_MoveSector_IgnoreObst_exp_1209_161533//agent.mp4


In [11]:
shutil.rmtree(os.path.join(load_path))